In [ ]:
# MIT License

# Copyright (c) 2017 jam1garner

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

#                     Description:
#                  ==================
# A Connect 4 AI that uses heuristic board definitions to evaluate
# a subsection of the game tree in order to decide which move is
# most likely to lead to a win. Made in less than an hour for a
# challenge/bet. Enjoy and hope you can learn something from it! 

from copy import deepcopy
import time

def printBoard(board):
    for i in range(7):
        print(i,end=' ')
    print()
    for row in board:
        for i in row:
            print(i,end=' ')
        print()

# Dictionaries to map iteration direction to end of range
xends = {-1 : -1, 1 : 7}
yends = {-1 : -1, 1 : 6}

# Checks if a board state is a win for a player
def isWin(board, x, y, player):
    directionPairs = [(1,0),(0,1),(1,1),(-1,1)] # 가로,세로,대각선
    for dx,dy in directionPairs:
        firstDirection = 0
        if dx == 0:
            xiter = [x for _ in range(7)]
        else:
            xiter = range(x + dx, xends[dx], dx)
        if dy == 0:
            yiter = [y for _ in range(7)]
        else:
            yiter = range(y + dy, yends[dy], dy)
        for xi,yj in zip(xiter,yiter):
            if board[yj][xi] == player:
                firstDirection+=1
            else:
                break
        secondDirection = 0
        if dx == 0:
            xiter = [x for _ in range(7)]
        else:
            xiter = range(x - dx, xends[-dx], -dx)
        if dy == 0:
            yiter = [y for _ in range(7)]
        else:
            yiter = range(y - dy, yends[-dy], -dy)
        for xi,yj in zip(xiter,yiter):
            if board[yj][xi] == player:
                secondDirection+=1
            else:
                break
        if secondDirection+firstDirection >= 3:
            return 1 # 이겼을 경우
    return 0

def connect3(board, x, y, player):
    directionPairs = [(1,0),(0,1),(1,1),(-1,1)] # 가로,세로,대각선
    ret = 0
    for dx,dy in directionPairs:
        firstDirection = 0
        block = 2 # 양쪽 끝이 막혔는가? 양쪽 다 막히면 2, 한쪽 1, 둘다 아니면 0
        if dx == 0:
            xiter = [x for _ in range(7)]
        else:
            xiter = range(x + dx, xends[dx], dx)
        if dy == 0:
            yiter = [y for _ in range(7)]
        else:
            yiter = range(y + dy, yends[dy], dy)
        for xi,yj in zip(xiter,yiter):
            if board[yj][xi] == player:
                firstDirection+=1
            elif board[yj][xi] == '-': # 한쪽 끝이 빈칸
                block -= 1
                break
            else: 
                break
        secondDirection = 0
        if dx == 0:
            xiter = [x for _ in range(7)]
        else:
            xiter = range(x - dx, xends[-dx], -dx)
        if dy == 0:
            yiter = [y for _ in range(7)]
        else:
            yiter = range(y - dy, yends[-dy], -dy)
        for xi,yj in zip(xiter,yiter):
            if board[yj][xi] == player:
                secondDirection+=1
            elif board[yj][xi] == '-': # 한쪽 끝이 빈칸
                block -= 1
                break                
            else :
                break
        if secondDirection+firstDirection >= 2:
            if dx == 0 : # 세로인 경우 가중치 down
                ret += 0.5*(2-block) # block이 2일 경우 <--> 양쪽 다 막혔을 경우 가중치 0 
            else:
                ret += 1 * (2-block) # block이 2일 경우 <--> 양쪽 다 막혔을 경우 가중치 0
                
    return ret  

def isWin2(board,player): # 현재 보드에 이길 수 있는 수가 2개 이상인가?
    count = 0
    for column in range(7):
        r,c = 5 - distanceFromBottom(board, column),column
        if isWin(board, c, r, player) and r > -1:
            count += 1
            if r > 0 :
                if isWin(board, c, r, player) and isWin(board, c, r-1, player) :
                    return 1 # 두 개가 위 아래로 연결된 경우
    if count > 1 :
        return 1
    else:
        return 0
                

# Return the distance from the bottom, this is used
# for knowing where the next piece will go
def distanceFromBottom(board, column):
    d = 0
    for i in range(5, -1, -1):
        if board[i][column] != '-':
            d += 1
        else:
            break
    return d

# Hueristically rate a board state this is
# somewhat arbitrary but it is weighted based on:
# * Whether each spot is a win
# * The distance from bottom that spot is
# * Defense > Offense (Weighted 4x as heavily)
# These values were tweaked iteratively based
# on tests against ~20 real world players, nothing
# scientific but is designed to do decently well
# based on human tendency.
def rateBoard(board):
    numberOfWinsX = 0
    for row in range(6):
        for column in range(7):
            if board[row][column] == '-':
                d = distanceFromBottom(board, column)
                numberOfWinsX += isWin(board, column, row, 'X') * (0.8 ** (5 - row - d))# 휴리스틱 scroe 매기기
    numberOfWinsO = 0    
    for row in range(6):
        for column in range(7):
            if board[row][column] == '-':
                d = distanceFromBottom(board, column)
                numberOfWinsO += isWin(board, column, row, 'O') * 4 * (0.8 ** (5 - row - d))# 휴리스틱 score 매기기
    return numberOfWinsX - numberOfWinsO

# Try all paths for a certain depth from a certain board
# state recursively, using the average hueristic rating 
# evaluation in order to pick one based on said average 
def recursivelyPickAiSpot(board, iterations, turn):
    if turn == 'X':
        turn = 'O'
    else:
        turn = 'X'
    if iterations <= 0:
        return rateBoard(board) # 끝 node에 도착했다면 rateBoard !
    avg = 0
    for column in range(7):
        boardCopy = deepcopy(board)
        r,c = 5 - distanceFromBottom(board, column),column # 들어갈 자리의 row,column
        if turn == 'O' and isWin(boardCopy, c, r, 'O'):  # O 차례, O가 이길수 있는 자리 : -4
            return -4
        elif turn == 'X' and isWin(boardCopy, c, r, 'X'):# X 차례, X가 이길수 있는 자리 : +1
            return 1
        boardCopy[r][c] = turn
        #printBoard(boardCopy)
        avg += recursivelyPickAiSpot(boardCopy, iterations - 1, turn)
    avg /= 7
    #print ('score :',avg,'iteration : ',iterations)
    return avg

# Pick the place to play, if a winning play exists pick it
# if there is a way to block an ensured loss pick it
# otherwise use the recursive column picking in order
# to decide which play is best
def pickAiSpot(board, iterations):
    
    for column in range(7):
        r,c = 5 - distanceFromBottom(board, column),column
        if board[r][c] == '-' and isWin(board, c, r, 'X'):
            return column  # 현재 특정 column에 X를 넣으면 X가 우승 --> 특정 column return (공격)
    for column in range(7):
        r,c = 5 - distanceFromBottom(board,column),column
        if board[r][c] == '-' and isWin(board, c, r, 'O'):
            return column # 현재 특정 column에 O를 넣으면 O가 우승 --> 특정 column return (수비)
        
        
    plays = [0 for i in range(7)] # [0,0,0,0,0,0,0] column별 score array
    
    for column in range(7):
        boardCopy = [row[:] for row in board]
        r,c = 5 - distanceFromBottom(board, column),column
        if r < 0: # 꽉찼다.
            plays[column] = -100000
        else:
            boardCopy[r][c] = 'X' # X를 넣는다면!
            plays[column] =  recursivelyPickAiSpot(boardCopy, iterations - 1, 'O')

    for column in range(7):
        boardCopy = [row[:] for row in board]
        r,c = 5 - distanceFromBottom(board, column),column
        if r > 0:
            boardCopy[r][c] = 'X'
            if isWin(boardCopy, c, r - 1, 'O'): # X를 넣고 그 위에 O를 넣으면 바로 O가 이길때
                plays[column] = -10000

    p = plays.copy()
    p.sort()
    return plays.index(p[-1]) # score를 sort해서 가장 큰값의 index return! --> score가 가장 큰 column을 return.

def rule(board):

    for column in range(7):
        r,c = 5 - distanceFromBottom(board, column),column
        if board[r][c] == '-' and isWin(board, c, r, 'X'):
            print('RULE#1 적용')
            return column  # RULE 1 : 만약 이길수 있는 상황이라면 돌을 놓는다.
        
    for column in range(7):
        r,c = 5 - distanceFromBottom(board,column),column
        if board[r][c] == '-' and isWin(board, c, r, 'O'):
            print('RULE#2 적용')
            return column # RULE 2 : 만약 상대방이 이길 상황이라면 돌을 놓아 수비한다.

    for column in range(7):
        r,c = 5 - distanceFromBottom(board, column),column
        boardCopy = [row[:] for row in board]
        boardCopy[r][c] = 'X'
        if board[r][c] == '-' and isWin2(boardCopy,'X'):
            print('RULE#1+ 적용')
            return column  # RULE 1+ : 만약 다음 차례에 이길수 있는 상황이라면 돌을 놓는다.
        
    for column in range(7):
        r,c = 5 - distanceFromBottom(board, column),column
        boardCopy = [row[:] for row in board]
        boardCopy[r][c] = 'O'
        if board[r][c] == '-' and isWin2(boardCopy,'O'):
            print('RULE#2+ 적용')
            return column  # RULE 2+ : 
        
    # Rules that guarantee a certain results

    plays = [0 for i in range(7)] # [0,0,0,0,0,0,0] column별 score array
    flag = 1
    for column in range(7):
        boardCopy = [row[:] for row in board]
        r,c = 5 - distanceFromBottom(board, column),column
        
        if r > 0:
            boardCopy[r][c] = 'X'
            if isWin(boardCopy, c, r - 1, 'O'): 
                if(flag):
                    print('RULE#3-1 적용')
                    flag = 0
                plays[column] += -10000 # RULE 3-1 : 상대방을 이길 수 있게 되는 위치 바로 밑에는 돌을 놓지 않는다.
    flag = 1            
    for column in range(7):
        boardCopy = [row[:] for row in board]
        r,c = 5 - distanceFromBottom(board, column),column
        
        if r > 0:
            #boardCopy[r][c] = 'X'
            if isWin(boardCopy, c, r - 1, 'X'):
                if(flag):
                    print('RULE#3-2 적용')
                    flag = 0
                plays[column] += -10000 # RULE 3-2 : 내가 이길 수 있게 되는 위치 바로 밑에는 돌을 놓지 않는다.
    flag = 1              
    for column in range(7):
        r,c = 5 - distanceFromBottom(board,column),column
        if board[r][c] == '-':
            sco = connect3(board, c, r, 'X')
            if(flag and sco > 0):
                print('RULE#4-1 적용',sco)
                flag = 0
            plays[column] += 100 * sco  # RULE 4-1 : AI가 3개 연결 (connect-3) 시킬 수 있는 수에 가중치 부여 (100점씩)
    flag = 1     
    for column in range(7):
        r,c = 5 - distanceFromBottom(board,column),column
        if board[r][c] == '-' :
            sco = connect3(board, c, r, 'O')
            if(flag and sco > 0):
                print('RULE#4-2 적용',sco)
                flag = 0
            plays[column] += 100 * sco  # RULE 4-2 : 상대방이 3개 연결 (connect-3)를 막을 수 있는 수에 가중치 부여 (100점씩)
            
    flag = 1
    for column in range(7):
        boardCopy = [row[:] for row in board]
        r,c = 5 - distanceFromBottom(board, column),column
        if r > 1:
            boardCopy[r][c] = 'X'
            if isWin(boardCopy, c, r - 2, 'X') : 
                if(flag):
                    print('RULE#5-1 적용')
                    flag = 0
                plays[column] += 1000 # RULE 5-1 : 내가 이길 수 있게 되는 위치 두 칸 밑에 가중치 부여.
                
    for column in range(7):
        r,c = 5 - distanceFromBottom(board, column),column
        scr = 0
        if r > 0:
            if c > 0:
                if board[r-1][c-1] == 'X': # 왼쪽 위
                    scr += 1
            elif c < 6:    
                if board[r-1][c+1] == 'X': # 오른쪽 위
                    scr += 1
        if r < 5:
            if c > 0:
                if board[r+1][c-1] == 'X': # 왼쪽 아래
                    scr += 1
            elif c < 6:
                if board[r+1][c+1] == 'X': # 오른쪽 아래
                    scr += 1
        plays[column] += scr*10
                
    flag = 1            
    for column in range(7):
        boardCopy = [row[:] for row in board]
        r,c = 5 - distanceFromBottom(board, column),column
        if r > 1:
            boardCopy[r][c] = 'X'
            if isWin(boardCopy, c, r - 2, 'O') : 
                if(flag):
                    print('RULE#5-2 적용')
                    flag = 0
                plays[column] += 1000 # RULE 5-2 : 상대방을 이길 수 있게 되는 위치 두 칸 밑에 가중치 부여.
                

    for column in range(7):    
        r,c = 5 - distanceFromBottom(board, column),column
        if r < 0: # 꽉찼다.
            plays[column] = -100000
        
    p = plays.copy()
    print(p)
    p.sort()
    if p[-1] == p[-2]: # RULE 6 : 중앙에 가까울수록 score 부여 (tie breaker)
        flag = 1
        for column in range(7):
            plays[column] -= (column-3)**2
        p = plays.copy()
        p.sort()
        if(flag):
            print('RULE#6 적용')
            flag = 0
    return plays.index(p[-1]) # score를 sort해서 가장 큰값의 index return! --> score가 가장 큰 column을 return.      

    # Rulse based scores
    

    

# User interface, kinda rough and doesn't verify
# user input so it isn't great but it does the job
# so long as the user doesn't try and cheat!
def main():
    board = [['-' for i in range(7)] for j in range(6)]
    winner = None
    
    print('AI 선공일 경우 0, 플레이어 선공일 경우 1을 입력해주세요!')
    first = input()
    if (first[0] == '0'):
        board[5][2] = 'X'
        
    while winner == None:
        printBoard(board)
        print('> ',end="")
        print('착수점 입력 :')
        i = input()
        print('Search Algorithm : 1 / Rule : 2')
        #f = input()
        #flag = int(f[0])
        flag = 2
        column = int(i[0])
        r,c = 5 - distanceFromBottom(board, column),column
        if isWin(board,c,r,'O'):
            winner = "O"
        board[5 - distanceFromBottom(board, column)][column] = 'O'
        if winner != None:
            break
        
        if (flag == 1):
            column = pickAiSpot(board[:], 5)
            r,c = 5 - distanceFromBottom(board, column),column
        elif (flag == 2):
            column = rule(board[:])
            r,c = 5 - distanceFromBottom(board, column),column
            
        if isWin(board,c,r,'X'):
            winner = "X"
        print('출력값 :',c )
        board[r][c] = 'X'

    printBoard(board)
    print('Winner is %s' % str(winner))

if __name__ == '__main__':
    main()
    